## Setup

To run these examples, you'll need an IonQ API key for an organization that has full beta/preview access to our v0.4 API.

As before, we'll make sure the API key is set up as an environment variable:

In [ ]:
import os
os.environ["IONQ_API_KEY"] = "YOUR API KEY HERE"

We'll use some built-in Python libraries as well (though you can work with the API directly in the command line or using other programming languages).

In [ ]:
import json
import requests

# API v0.4

We're in the process of launching API v0.4 and upgrading our platform and integrations from API v0.3. We'll continue to support v0.3, and much of v0.4 will look similar, but there are some differences.

Using the API is not required (you may find that an SDK is more convenient, and is sufficient for everything you need to do), but knowing what it does can provide a deeper understanding of how our SDK integrations and our systems work, and can provide access to some capabilities that aren't fully integrated into all SDKs.

## Check access and credentials

To check if we can connect to the API, we'll send a `GET` request to an endpoint that doesn't require authorization. If the response code is 200, this worked.

In [ ]:
response = requests.get("https://api.ionq.co/v0.4/backends")

In [ ]:
response

You might also see examples with slightly different formatting, as in our docs. Here, `requests.get(...)` is equivalent to `requests.request("GET"...)`.

In [ ]:
response = requests.request("GET", "https://api.ionq.co/v0.4/backends")

In [ ]:
response

Next, set up the API request headers, which will include our authentication (API key). Here we're retrieving the key that was stored as an environment variable above (or external to this notebook), but you could also just put your API key here. We'll use the same headers for all requests that require authentication.

In [ ]:
headers = {
    "Authorization": f"apiKey {os.getenv('IONQ_API_KEY')}",
    "Content-Type": "application/json",
}

Look at the headers (note that this prints your API key):

In [ ]:
headers

Send a request that requires an API key - in this case, retrieving our recent jobs. We'll look at the content of the response later, but for now, a code of 200 indicates that the connection was successful.

In [ ]:
response = requests.get("https://api.ionq.co/v0.4/jobs", headers=headers)

In [ ]:
response

In [ ]:
response.json()

## API key info

The "who am I" API endpoint returns information about the API key that was used to send the request. This can be especially useful if you're managing multiple API keys from different projects, and you want to confirm that jobs submitted with this API key will be sent to the correct project.

In [ ]:
response_whoami = requests.get(
    f"https://api.ionq.co/v0.4/whoami",
    headers=headers
)

View the response as json (effectively a Python dictionary):

In [ ]:
response_whoami.json()

This gives us a unique identifier for the key, the name we gave the key when we created it, and the unique identifier for the project it's tied to.

Check the cloud console to see which project this key corresponds to.

In [ ]:
print(f"https://cloud.ionq.com/projects/{response_whoami.json()['project_id']}")

## Jobs

The API can be used to submit a job, retrieve a job's results, and retrieve other information about the job.

### Create a job

[Creating a job]() involves sending a POST request to `https://api.ionq.co/v0.4/jobs` with a payload containing the circuit, target backend, and job settings.

Set up the job data in the required format. Additional information on the available fields is included in [the documentation for this API endpoint]() and more details about the circuit format can be found in [our guide to building circuits in the API]() (note that this page has not yet been updated for API v0.4, but the circuit format has not changed).

Note that when you use `extra_query_params` when running a job in Qiskit or Cirq, the arguments included there are put into this request payload.

In [ ]:
job_data = {
    "name": "API example - sim",
    "type": "ionq.circuit.v1",
    
    "backend": "simulator",
    "noise": {"model": "ideal"},
    
    "input": {
        "qubits": 2,
        "gateset": "qis",
        "circuit": [
            {"gate": "h", "target": 0},
            {"gate": "x", "target": 1, "control": 0}
        ]
    },
}

Send the request:

In [ ]:
response_create_job = requests.post(
    "https://api.ionq.co/v0.4/jobs",
    headers=headers,
    json=job_data
)

The response (if the job was submitted successfully) has the job ID. A response code of 404 usually means you don't have the right credentials, access, permissions, etc. while a code of 400 usually means an issue with the syntax or structure. 201 means the job was submitted successfully, though it can still fail during subsequent steps.

In [ ]:
response_create_job

Look at the response body to get the status and job ID:

In [ ]:
response_create_job.json()

The job ID is populated as soon as the job is submitted, but we have to send another request to get the updated status for this job. Save the job ID, from the create job response or the cloud console:

In [ ]:
job_id_sim = response_create_job.json()['id']
print(job_id_sim)

### Get a job

This is a `get` request to the endpoint `jobs/MY_JOB_ID`. We'll put the job ID into the URL and include the headers with the API key, but there is no data or other payload in this request.

In [ ]:
response_get_job = requests.get(
    f"https://api.ionq.co/v0.4/jobs/{job_id_sim}",
    headers=headers
)

First look at the status:

In [ ]:
response_get_job.json()['status']

The full response contains a lot of other information about the job - some fields are based on your submission, some were populated by IonQ's cloud platform. Some job-related information, like the job cost and results, uses different API endpoints (see below).

In [ ]:
response_get_job.json()

### Get a job's result probabilities

The result probabilities can be requested using a specific URL. This is included in the response above, but you can also plug the job ID into the URL structure.

In [ ]:
job_result_url = "https://api.ionq.co" + response_get_job.json()['results']['probabilities']['url']

In [ ]:
job_result_url = f"https://api.ionq.co/v0.4/jobs/{job_id_sim}/results/probabilities"

In [ ]:
print(job_result_url)

Send a GET request to this URL:

In [ ]:
response_get_result = requests.get(
    job_result_url,
    headers=headers
)

This response is just a dictionary containing the probabilities for each state:

In [ ]:
response_get_result.json()

In the future, `results` may include additional fields depending on the job type and result format.

### Submit a job to a QPU

QPU submission is similar to running on a simulator:

In [ ]:
job_data_qpu = {
    "name": "API example - Aria 1",
    "type": "ionq.circuit.v1",

    "backend": "qpu.aria-1",
    "shots": 100,
    
    "input": {
        "qubits": 2,
        "gateset": "qis",
        "circuit": [
            {"gate": "h", "target": 0},
            {"gate": "x", "target": 1, "control": 0}
        ]
    },
}

In [ ]:
response_create_job_qpu = requests.post(
    "https://api.ionq.co/v0.4/jobs",
    headers=headers,
    json=job_data_qpu
)

In [ ]:
response_create_job_qpu.json()

In [ ]:
job_id_qpu1 = response_create_job_qpu.json()['id']

As before, let's set up a request to get the job status. We'll confirm that the job went from "submitted" to "ready" (queued).

In [ ]:
response_get_job_qpu = requests.get(
    f"https://api.ionq.co/v0.4/jobs/{job_id_qpu1}",
    headers=headers
)

In [ ]:
response_get_job_qpu.json()['status']

The full response for this job includes a lot of information, similar to the simulator job example above - but it doesn't include the result URL, because the result doesn't exist yet. Instead it shows that the result is `None`, for now.

In [ ]:
response_get_job_qpu.json()

If we try to retrieve the probabilities now, using the job ID and the standard result URL format, we'll get a 404 error because there was no result found at that URL.

In [ ]:
response_get_result_qpu = requests.get(
    f"https://api.ionq.co/v0.4/jobs/{job_id_qpu1}/results/probabilities",
    headers=headers
)

In [ ]:
response_get_result_qpu.json()

### Retrieve QPU job results

Let's check the cloud console and find a job ID from a completed QPU job, like the Qiskit example from earlier (you can retrieve any job using the API, regardless of how it was submitted):

In [ ]:
job_id_qpu2 = "YOUR JOB ID HERE"

In [ ]:
response_get_result_qpu2 = requests.get(
    f"https://api.ionq.co/v0.4/jobs/{job_id_qpu2}/results/probabilities",
    headers=headers
)

As in the simulator example, this just returns a list of the measured states and their probabilities:

In [ ]:
response_get_result_qpu2.json()

#### Probabilities and shot counts

Job results are _stored_ as probabilities - not only for ideal simulation jobs where the actual result is the calculated probability, but also for QPU and noisy simulation jobs where the original result was a set of shot counts.

We can get the total number of shots from the job info:

In [ ]:
response_get_job_qpu2 = requests.get(
    f"https://api.ionq.co/v0.4/jobs/{job_id_qpu2}",
    headers=headers
)

Then we can reconstruct a dictionary containing the number of shots:

In [ ]:
shots = response_get_job_qpu2.json()['shots']

probs_dict = response_get_result_qpu2.json()

counts_dict = dict()
for key, val in probs_dict.items():
    counts_dict[key] = int(val * shots)

In [ ]:
counts_dict

### Get a job's cost

For QPU jobs, you can also see how much a job cost (or would have cost):

In [ ]:
response_job_cost_qpu2 = requests.get(
    f"https://api.ionq.co/v0.4/jobs/{job_id_qpu2}/cost",
    headers=headers
)

If this job actually ran, this will show the estimated and actual cost:

In [ ]:
response_job_cost_qpu2.json()

We can also request the cost for the other job from earlier:

In [ ]:
response_job_cost_qpu1 = requests.get(
    f"https://api.ionq.co/v0.4/jobs/{job_id_qpu1}/cost",
    headers=headers
)

In [ ]:
response_job_cost_qpu1.json()

### Dry run

Submitting a job with the `dry_run` option will perform the usual job submission process but will not actually add the job to the queue or run it. You can use this to test jobs and workflows (in addition to simulation and noisy simulation) as well as to get predicted job cost and timing information.

In [ ]:
job_data_dryrun = {
    "name": "API example - dry run",
    "type": "ionq.circuit.v1",

    # Don't actually submit to the QPU
    "dry_run": True,

    "backend": "qpu.aria-1",
    "shots": 1000,
    # Optionally, turn off debiasing
    #"settings": {"error_mitigation": {"debiasing": False}},
    
    "input": {
        "qubits": 2,
        "gateset": "qis",
        "circuit": [
            {"gate": "h", "target": 0},
            {"gate": "x", "target": 1, "control": 0}
        ]
    },
}

Submit the job:

In [ ]:
response_create_job_dryrun = requests.post(
    "https://api.ionq.co/v0.4/jobs",
    headers=headers,
    json=job_data_dryrun
)

In [ ]:
response_create_job_dryrun.json()

In [ ]:
job_id_dryrun = response_create_job_dryrun.json()['id']

The job status should change to "completed" instead of "ready", since it's not actually entering the queue. This should happen quickly.

In [ ]:
response_get_job_dryrun = requests.get(
    f"https://api.ionq.co/v0.4/jobs/{job_id_dryrun}",
    headers=headers
)

In [ ]:
response_get_job_dryrun.json()['status']

The response is similar to a queued job (with some information, like the submission time and predicted execution duration, already populated). However, this job is _completed_ with no results.

In [ ]:
response_get_job_dryrun.json()

Check the predicted execution duration, which is measured in milliseconds. This doesn't give the full submission-to-results job duration (it doesn't include the queue or all pre/post-processing steps) or the direct billable time, but it can give an estimate of roughly how long the actual QPU execution part of the job will take.

In [ ]:
response_get_job_dryrun.json()['predicted_execution_duration_ms']

We can also request the cost:

In [ ]:
response_job_cost_dryrun = requests.get(
    f"https://api.ionq.co/v0.4/jobs/{job_id_dryrun}/cost",
    headers=headers
)

In [ ]:
response_job_cost_dryrun.json()

This gives the estimated cost for the job, in credit expressed in USD.

### Estimate cost without a circuit

You can also get a cost estimate without actually preparing the job and submitting a dry run. The [estimate](https://docs.ionq.com/api-reference/v0.4/jobs/get-job-estimate) API endpoint gives a cost estimate based on gate and shot counts, backend, and error mitigation setting.

In [ ]:
estimate = {
    "backend": "qpu.aria-1",
    "qubits": 25,
    "shots": 1000,
    "1q_gates": 400,
    "2q_gates": 150,
    "error_mitigation": True
}

In [ ]:
response_estimate = requests.get(
    "https://api.ionq.co/v0.4/jobs/estimate",
    headers=headers,
    params=estimate
)

In [ ]:
response_estimate.json()

You can also do this in the docs.

## Wrap-up

While API v0.4 is currently in beta and not all endpoints are fully documented, it will be officially released soon. More job types, result formats, and settings will also become available in the future. For now, we'd appreciate your feedback on 

https://docs.ionq.com/api-reference/v0.4/introduction